# 项目：整理Netflix电影演员评分数据

## 分析目标

此数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

本实战项目的目的在于练习整理数据，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了截止至2022年7月美国地区可观看的所有Netflix电视剧及电影数据。数据集包含两个数据表：`titles.csv`和`credits.csv`。

`titles.csv`包含电影及电视剧相关信息，包括影视作品ID、标题、类型、描述、流派、IMDB（一个国外的在线评分网站）评分，等等。`credits.csv`包含超过7万名出现在Netflix影视作品的导演及演员信息，包括名字、影视作品ID、人物名、演职员类型（导演/演员）等。

`titles.csv`每列的含义如下：
- id：影视作品ID。
- title：影视作品标题。
- show_type：作品类型，电视节目或电影。
- description：简短描述。
- release_year：发布年份。
- age_certification：适龄认证。
- runtime：每集电视剧或电影的长度。
- genres：流派类型列表。
- production_countries：出品国家列表。
- seasons：如果是电视剧，则是季数。
- imdb_id：IMDB的ID。
- imdb_score：IMDB的评分。
- imdb_votes：IMDB的投票数。
- tmdb_popularity：TMDB的流行度。
- tmdb_score：TMDB的评分。

`credits.csv`每列的含义如下：
- person_ID：演职员ID。
- id：参与的影视作品ID。
- name：姓名。
- character_name：角色姓名。
- role：演职员类型，演员或导演。

In [1]:
import pandas as pd
import numpy as np

In [2]:
df1=pd.read_csv("E:\\A_数据分析资料\\credits.csv")

In [3]:
df2=pd.read_csv("E:\\A_数据分析资料\\titles.csv")

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5850 non-null   object 
 1   title                 5849 non-null   object 
 2   type                  5850 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5850 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5850 non-null   int64  
 7   genres                5850 non-null   object 
 8   production_countries  5850 non-null   object 
 9   seasons               2106 non-null   float64
 10  imdb_id               5447 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score            5539 non-null   float64
dtypes: float64(5), int64(

分析目的：整理不同流派影视作品，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员
分析的列
genres：流派类型列表、role：演职员类型，演员或导演、imdb_score：IMDB的评分
键
id：影视作品ID、person_ID：演职员ID

清理credits.csv演员、演员id缺失的列,charcter演出角色姓名缺失不影响挖掘高评分作品的演员

1、评估数据

In [6]:
df1.sample(10)

,person_id,id,name,character,role
27318,204824,tm232855,Nonie Buencamino,JJ,ACTOR
57742,86306,tm946360,Maria Broom,Regina Maxwell,ACTOR
3910,103836,tm55100,Aditya Srivastava,Terrorist,ACTOR
61645,133429,ts89182,Enno Cheng,Liu Zao Yun,ACTOR
35037,263922,ts81526,François Hollande,Himself,ACTOR
67310,1249548,tm1094217,Jordan Tartakow,Chickenhare (voice),ACTOR
21248,66333,ts38090,Evan Williams,The Knight of Lorraine,ACTOR
49504,194743,tm446609,Steve Bannon,Self - Former Chief Executive Trump Campaign (...,ACTOR
42577,298189,ts82887,Mai Shiraishi,Tsuki Ayako,ACTOR
54139,1184679,tm483997,Motsi Tekateka,DDM Official,ACTOR


根据分析目的，提取出role只包含演员的行

In [7]:
df1_actor=df1.query('role=="ACTOR"')

In [8]:
#创建一个副本，区别清理前后的df
df1_actor_clean=df1_actor.copy()

In [9]:
df1_actor_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73251 entries, 0 to 77799
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  73251 non-null  int64 
 1   id         73251 non-null  object
 2   name       73251 non-null  object
 3   character  68029 non-null  object
 4   role       73251 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [10]:
#person_id转化成文本
df1_actor_clean["person_id"]=df1_actor_clean["person_id"].astype(str)

In [11]:
df2.sample(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
4160,ts236197,StarBeam,SHOW,"Zoey is excited to tackle second grade, but wh...",2020,TV-Y,13,[],['CA'],4.0,NaN,NaN,NaN,6.803,6.9
1883,ts86296,Sunderland 'Til I Die,SHOW,This docuseries follows English soccer club Su...,2018,TV-MA,41,"['documentation', 'sport']",['GB'],2.0,tt8914684,8.1,8113.0,5.099,8.0
1866,tm366567,When We First Met,MOVIE,Noah spends the perfect first night with the g...,2018,PG-13,97,"['fantasy', 'comedy', 'romance']",['US'],NaN,tt5783956,6.4,48885.0,15.879,6.6
2647,tm357272,Barbra: The Music ... The Mem'ries ... The Magic!,MOVIE,Iconic songstress Barbra Streisand culminates ...,2017,NaN,108,"['documentation', 'music']",['US'],NaN,tt7588850,7.5,448.0,4.214,7.4
5466,tm975741,Octonauts: The Ring of Fire,MOVIE,"In this film's adventure, a volcano erupts sud...",2021,G,72,['animation'],"['CN', 'GB']",NaN,tt13779166,6.4,137.0,5.576,6.8
3754,tm485801,So My Grandma's a Lesbian!,MOVIE,A promising young lawyer sees her plans to wed...,2019,NaN,94,"['comedy', 'romance', 'european']","['ES', 'PT']",NaN,tt9695258,4.3,1122.0,6.736,5.3
2799,tm327951,The Doll 2,MOVIE,A couple who devastated after losing their onl...,2017,NaN,116,"['horror', 'thriller']",['ID'],NaN,tt6409782,5.1,550.0,5.769,5.6
2359,tm370628,Fred Armisen: Standup for Drummers,MOVIE,"For an audience of drummers, comedian Fred Arm...",2018,NaN,65,"['music', 'comedy']",['US'],NaN,tt7924798,5.8,1294.0,4.256,6.2
440,tm71119,Delta Farce,MOVIE,Three bumbling Army reservists are hustled ont...,2007,PG-13,90,"['comedy', 'action']",['US'],NaN,tt0800003,3.6,11177.0,10.367,5.0
4301,tm926528,AK vs AK,MOVIE,"After a public spat with a movie star, a disg...",2020,NaN,108,"['drama', 'action', 'crime', 'thriller', 'come...",['IN'],NaN,tt11651796,6.9,14195.0,3.757,7.3


In [12]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5850 non-null   object 
 1   title                 5849 non-null   object 
 2   type                  5850 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5850 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5850 non-null   int64  
 7   genres                5850 non-null   object 
 8   production_countries  5850 non-null   object 
 9   seasons               2106 non-null   float64
 10  imdb_id               5447 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score            5539 non-null   float64
dtypes: float64(5), int64(

imdb_score是分析的重点，这列缺失的行可以去掉 

In [13]:
#创建一个副本
df2_clean=df2.copy()

In [14]:
#dropna方法，传入subset参数，删除imdb_score为na的行
df2_clean=df2_clean.dropna(subset=["imdb_score"])

In [15]:
df2_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5368 entries, 1 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5368 non-null   object 
 1   title                 5368 non-null   object 
 2   type                  5368 non-null   object 
 3   description           5363 non-null   object 
 4   release_year          5368 non-null   int64  
 5   age_certification     3033 non-null   object 
 6   runtime               5368 non-null   int64  
 7   genres                5368 non-null   object 
 8   production_countries  5368 non-null   object 
 9   seasons               1939 non-null   float64
 10  imdb_id               5368 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5293 non-null   float64
 14  tmdb_score            5145 non-null   float64
dtypes: float64(5), int64(2), o

In [16]:
df2_clean.genres

1                                ['drama', 'crime']
2       ['drama', 'action', 'thriller', 'european']
3                   ['fantasy', 'action', 'comedy']
4                                 ['war', 'action']
5                            ['comedy', 'european']
                           ...                     
5843                 ['romance', 'comedy', 'drama']
5845                           ['romance', 'drama']
5846                                      ['drama']
5847                                     ['comedy']
5849              ['family', 'animation', 'comedy']
Name: genres, Length: 5368, dtype: object

In [17]:
df2_clean.genres[1]
#genres列形式是列表，实际是字符串

"['drama', 'crime']"

In [18]:
#eval(x)把字符串列表转换成列表，并将转换后的结果赋值给genres
df2_clean.genres=df2_clean.genres.apply(lambda x:eval(x))

In [19]:
df2_clean=df2_clean.explode("genres")

In [20]:
df2_clean.production_countries[1]

1    ['US']
1    ['US']
Name: production_countries, dtype: object

In [21]:
df2_clean.production_countries=df2_clean.production_countries.apply(lambda x:eval(x))

In [22]:
df2_cleaned=df2_clean.explode("production_countries")

In [23]:
df2_cleaned

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,thriller,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5846,tm898842,C/O Kaadhal,MOVIE,A heart warming film that explores the concept...,2021,NaN,134,drama,NaN,NaN,tt11803618,7.7,348.0,NaN,NaN
5847,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021,NaN,7,family,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021,NaN,7,animation,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000


观察影视作品，演员表是否有重复值

In [24]:
df1_actor_clean.duplicated().sum()

0

In [25]:
df2_cleaned.duplicated().sum()

0

评估，清洗完的数据df1_actor_clean，df2_cleaned
通过影视作品id拼接这两个df,出于分析的目的，需要用内连接inner

In [26]:
df3=pd.merge(df1_actor_clean,df2_cleaned,on=["id"])

In [27]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262616 entries, 0 to 262615
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   person_id             262616 non-null  object 
 1   id                    262616 non-null  object 
 2   name                  262616 non-null  object 
 3   character             249643 non-null  object 
 4   role                  262616 non-null  object 
 5   title                 262616 non-null  object 
 6   type                  262616 non-null  object 
 7   description           262596 non-null  object 
 8   release_year          262616 non-null  int64  
 9   age_certification     176896 non-null  object 
 10  runtime               262616 non-null  int64  
 11  genres                262614 non-null  object 
 12  production_countries  261657 non-null  object 
 13  seasons               44537 non-null   float64
 14  imdb_id               262616 non-null  object 
 15  

In [28]:
df4=df3.groupby(["genres","id","person_id"])["imdb_score"].mean()

In [29]:
df4=df4.reset_index()

In [30]:
#ascending参数设置降序或者升序
df4=df4.sort_values("imdb_score",ascending=False).reset_index()

重置索引，删除index列

In [31]:
df4.drop("index",axis=1,inplace=True)

In [32]:
df4

,genres,person_id,imdb_score
0,thriller,20552,9.5
1,drama,37752,9.5
2,crime,20552,9.5
3,thriller,37752,9.5
4,drama,20552,9.5
...,...,...,...
168876,scifi,1431151,1.5
168877,scifi,1431135,1.5
168878,scifi,1431133,1.5
168879,scifi,1430247,1.5


In [33]:
df4.query('genres=="action"')

,genres,person_id,imdb_score
9,action,12790,9.3
22,action,1303,9.3
25,action,21033,9.3
27,action,336830,9.3
28,action,86591,9.3
...,...,...,...
168819,action,65927,1.7
168823,action,237857,1.7
168824,action,881371,1.7
168827,action,918609,1.7


In [34]:
genres_max=df4.groupby(["genres"])["imdb_score"].max()

In [35]:
genres_max

genres
action           9.3
animation        9.3
comedy           9.2
crime            9.5
documentation    9.1
drama            9.5
european         8.9
family           9.3
fantasy          9.3
history          9.1
horror           9.0
music            8.8
reality          8.9
romance          9.2
scifi            9.3
sport            9.1
thriller         9.5
war              8.8
western          8.9
Name: imdb_score, dtype: float64

In [36]:
df5=pd.merge(genres_max,df4,on=["genres","imdb_score"])

In [37]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   genres      136 non-null    object 
 1   imdb_score  136 non-null    float64
 2   person_id   136 non-null    object 
dtypes: float64(1), object(2)
memory usage: 3.3+ KB


In [38]:
df5

,genres,imdb_score,person_id
0,action,9.3,12790
1,action,9.3,1303
2,action,9.3,21033
3,action,9.3,336830
4,action,9.3,86591
...,...,...,...
131,war,8.8,47019
132,western,8.9,28180
133,western,8.9,93017
134,western,8.9,28166


In [40]:
df5.to_csv("E:\\A_处理文件\\整理后的按类型imdb_score最高的影视作品、演员.csv",encoding='utf-8-sig')